In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0, 1, 2, 3
%matplotlib notebook

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import time
import os
import copy
from skimage import io, transform
from torch.utils.data import DataLoader
from PIL import Image
import json
from math import pi
from copy import copy




In [ ]:
run_name = 'test_runs02/000_double_layer_very_narrow'
model_name = 'resnet50'
batch_size = 256
epochs = 20
debug = False
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_split = 0.8


# validation_split = .2
# shuffle_dataset = True
# random_seed= 42
# https://stackoverflow.com/questions/50544730/split-dataset-train-and-test-in-pytorch-using-custom-dataset


In [ ]:
def plot_history(history):
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111)
    
    ax.plot(np.array(history['loss'])[::2], label='loss1')
    ax.plot(np.array(history['loss'])[1::2], label='loss1')
    ax.plot(history['accuracy'], label='accuracy')
    fig.legend(loc=2,  title="Training")
    fig.show()
    ax.grid(True)
    

In [ ]:
if not os.path.exists(run_name):
    os.makedirs(run_name)
    
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
#         transforms.ColorJitter(brightness=.2, contrast=0.2, saturation=0.2, hue=0),
#         transforms.RandomAffine(degrees=[-45,45], translate=None, scale=[.8, 1.2], shear=None),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

root = '/home/mate/dataset/EuroSatRGB/images'
full_dataset = datasets.ImageFolder(root, transform=data_transforms['test']) # does not support augmentation yet
train_size = int(train_split * len(full_dataset))
test_size = len(full_dataset) - train_size
dataset_sizes = {'train': train_size, 'test': test_size}
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_dataset.dataset = copy(full_dataset)
train_dataset.transforms = data_transforms['train']


image_datasets = {'train': train_dataset,
                  'test': test_dataset}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'test']}

#------
# image_datasets = {x: datasets.ImageFolder(root, transform=data_transforms[x])
#                   for x in ['train', 'test']}

# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

# indices = {x: list(range(dataset_sizes[x])) for x in ['train', 'test']}

# split = int(np.floor(validation_split * dataset_sizes['test']))
# if shuffle_dataset :
#     np.random.seed(42)
#     np.random.shuffle(indices)
# train_indices, val_indices = indices['train'][split:], indices['test'][:split]

# # Creating PT data samplers and loaders:
# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)

# 


In [ ]:
def train_model(model, criterion, optimizer, scheduler,
                num_epochs=25):
    
    since = time.time()
    
    history = {'loss': [],
               'accuracy': []}
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    else:
                        output = (outputs>0.5).float()
                        number_of_corrects = (output.argmax(dim=1) == labels).float().sum()
                        running_corrects += number_of_corrects
                        

                # statistics
                running_loss += loss.item() * inputs.size(0)
                if debug:
                    break
            
            epoch_loss = float(running_loss / dataset_sizes[phase]) 
            history['loss'].append(epoch_loss)
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            
            if phase == 'test':
                epoch_accuracy = float(running_corrects / dataset_sizes[phase])
                history['accuracy'].append(epoch_accuracy)
                print('{} Accuracy: {:.4f}'.format(phase, epoch_accuracy))
                with open(os.path.join(run_name, 'history.json'), 'w') as fout:
                    json.dump(history, fout)

            

            
            if not debug:
                torch.save(model, os.path.join(run_name, model_name + '_{}.pth'.format(epoch)))
    

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    

    return history, model

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    for param in model.parameters():
        param.requires_grad = feature_extracting

In [ ]:
# if model_name == 'resnet18':
#     model_ft = models.resnet18(pretrained=True)
# elif model_name == 'resnet50':
#     model_ft = models.resnet50(pretrained=True)
# elif model_name == 'resnet152':
#     model_ft = models.resnet152(pretrained=True)
# else:
#     raise ValueError('Unknown torchvision model name.')

model_ft = models.resnet50(pretrained=True)
set_parameter_requires_grad(model_ft, False)

In [ ]:
class Multilayer(nn.Module):
    def __init__(self, features_in, hidden=1024):
        super(Multilayer, self).__init__()
        self.layer1 = nn.Linear(features_in, hidden)
#         self.dropout = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(hidden, 10)
        
    def forward(self, x):
        x = self.layer1(x)
#         x = self.dropout(x)
        x = self.layer2(x)
        return x

In [ ]:
num_ftrs = model_ft.fc.in_features
model_ft.fc = Multilayer(num_ftrs, hidden=10)
# model_ft.fc = nn.Linear(num_ftrs, 10)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_ft = nn.DataParallel(model_ft)

model_ft = model_ft.to(device)
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=1e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
history, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)

In [ ]:
plot_history(history)
plt.show()

In [ ]:
set_parameter_requires_grad(model_ft, True)
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=1e-5)
history2, model_ft2 = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)

In [ ]:
plot_history(history2)